In [1]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
import sklearn
from Util.data_loader import label_image
from Util.extractors import create_feature_bow, extract_sift_features, extract_orb_features, kmean_bow
import pickle
import argparse
import os
from sklearn.model_selection import LeaveOneOut

In [6]:
DATA_AUG_TRAIN_PATH = "../../../Dataset/EarVN1.0 dataset/aug_dataset_300/train"
DATA_AUG_TRAIN_600_PATH = "../../../Dataset/EarVN1.0 dataset/aug_dataset_600/train"
DATA_AUG_TEST_PATH = "../../../Dataset/EarVN1.0 dataset/split_dataset/test"

In [3]:
def extract_feature(path, num_cluster=60):
    data_train, label, label2id = label_image(path)
    
    image_desctiptors = extract_sift_features(data_train)
    
    all_descriptors = []
    for descriptor in image_desctiptors:
        if descriptor is not None:
            for des in descriptor:
                all_descriptors.append(des)
    
    BoW = kmean_bow(all_descriptors, num_cluster)
    
    X_features = create_feature_bow(
        image_desctiptors, BoW, num_cluster
    )
    
    return X_features, label, label2id

In [4]:
X_features, label, label2id = extract_feature(DATA_AUG_TRAIN_PATH)

[ WARN:0@37.443] global /Users/runner/work/opencv-python/opencv-python/opencv_contrib/modules/xfeatures2d/misc/python/shadow_sift.hpp (15) SIFT_create DEPRECATED: cv.xfeatures2d.SIFT_create() is deprecated due SIFT tranfer to the main repository. https://github.com/opencv/opencv/issues/16736


In [7]:
X_features_test, label_test, label2id_test = extract_feature(DATA_AUG_TEST_PATH)

In [8]:
# build model:
model_svm = sklearn.svm.SVC(
    kernel = "poly",
    C = 35, 
    gamma = 0.01,
    random_state = 0
)

In [9]:
model_svm.fit(X_features, label)
print("score on training set params: ", model_svm.score(X_features, label))
print("score on testing set params: ", model_svm.score(X_features_test, label_test))

score on training set params:  0.678778386844166
score on testing set params:  0.014069644741470278


In [10]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(10)
neigh.fit(X_features, label)

print("score on training set params: ", neigh.score(X_features, label))
print("score on testing set params: ", neigh.score(X_features_test, label_test))

score on training set params:  0.4629600626468285
score on testing set params:  0.009848751319029195


In [11]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=80, max_depth=70, random_state=0)
clf.fit(X_features, label)

print("score on training set params: ", clf.score(X_features, label))
print("score on testing set params: ", clf.score(X_features_test, label_test))

score on training set params:  0.9962859380243875
score on testing set params:  0.011255715793176222
